In [243]:
import pandas as pd
import numpy as np
import time
import warnings
warnings.filterwarnings('ignore')

In [244]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
import pickle
import matplotlib.pyplot as plt

In [245]:
# Using chi2 (a statistical measure for finding independence between categorical feature and a target variable)
def selectkbest(indep_x, dep_y, n):
    test = SelectKBest(score_func = chi2, k = n)
    fit1 = test.fit(indep_x, dep_y)
    selectk_features = fit1.transform(indep_x)
    return selectk_features

In [246]:
def split_scaler(indep_x, dep_y):
    x_train, x_test, y_train, y_test = train_test_split(indep_x, dep_y, test_size = 0.30, random_state = 0)
    sc = StandardScaler()
    x_train = sc.fit_transform(x_train)
    x_test = sc.transform(x_test)
    return x_train, x_test, y_train, y_test 

In [247]:
def cm_prediction(classifier, x_test):
    from sklearn.linear_model import LogisticRegression
   
    y_pred = classifier.predict(x_test)
    # Making Confusion Matrix
    from sklearn.metrics import confusion_matrix
    cm = confusion_matrix(y_test, y_pred)
    
    from sklearn.metrics import accuracy_score
    accuracy = accuracy_score(y_test, y_pred)
    
    from sklearn.metrics import classification_report
    report = classification_report(y_test, y_pred)
    return classifier, accuracy, report, x_test, y_test, cm

In [248]:
def logistic(x_train,y_train,x_test):       
    # Fitting K-NN to the Training set
    from sklearn.linear_model import LogisticRegression
    classifier = LogisticRegression(random_state=0)
    classifier.fit(x_train, y_train)
    classifier,accuracy,report,x_test,y_test,cm = cm_prediction(classifier,x_test)
    return  classifier,accuracy,report,x_test,y_test,cm   

In [249]:
def svm_linear(x_train, y_train, x_test):
    from sklearn.svm import SVC
    classifier = SVC(kernel = 'linear', random_state = 0)
    classifier.fit(x_train, y_train)
    classifier, accuracy, report, x_test, y_test, cm = cm_prediction(classifier, x_test) 
    return  classifier,accuracy,report,x_test,y_test,cm    

In [250]:
def svm_nl(x_train, y_train, x_test):
    from sklearn.svm import SVC
    classifier = SVC(kernel = 'rbf', random_state = 0)
    classifier.fit(x_train, y_train)
    classifier, accuracy, report, x_test, y_test, cm = cm_prediction(classifier, x_test) 
    return  classifier,accuracy,report,x_test,y_test,cm    

In [251]:
def knn(x_train, y_train, x_test):
    from sklearn.neighbors import KNeighborsClassifier
    classifier = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
    classifier.fit(x_train, y_train)
    classifier, accuracy, report, x_test, y_test, cm = cm_prediction(classifier, x_test)
    return  classifier,accuracy,report,x_test,y_test,cm   

In [252]:
def naive(x_train, y_train, x_test):
    from sklearn.naive_bayes import GaussianNB
    classifier = GaussianNB()
    classifier.fit(x_train, y_train)
    classifier, accuracy, report, x_test, y_test, cm = cm_prediction(classifier, x_test)
    return  classifier,accuracy,report,x_test,y_test,cm   

In [2]:
def decision(x_train, y_train, x_test):
    from sklearn.tree import DecisionTreeClassifier
    classifier = DecisionTreeClassifier(criterion = 'entropy', random_state = 0)
    classifier.fit(x_train, y_train)
    classifier, accuracy, report, x_test, y_test, cm = cm_prediction(classifier, x_test) 
    return  classifier,accuracy,report,x_test,y_test,cm   

In [3]:
def random(x_train, y_train, x_test):
    from sklearn.ensemble import RandomForestClassifier
    classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
    classifier.fit(x_train, y_train)
    classifier, accuracy, report, x_test, y_test, cm = cm_prediction(classifier, x_test) 
    return  classifier,accuracy,report,x_test,y_test,cm   

In [255]:
def selectk_classification(acclog, accsvml, accsvmnl, accknn, accnaive, accdeci, accrand):
    dataframe = pd.DataFrame(index = ['ChiSquare'], columns = ['Logistic', 'SVML', 'SVMNL', 'KNN', 'Naive', 'Decision', 'Random'])
    for number, item in enumerate(dataframe.index):       # Enumerate - it is a counter that adds index to each items in an iterable
        dataframe['Logistic'][item] = acclog[number]
        dataframe['SVML'][item] = accsvml[number]
        dataframe['SVMNL'][item] = accsvmnl[number]
        dataframe['KNN'][item] = accknn[number]
        dataframe['Naive'][item] = accnaive[number]
        dataframe['Decision'][item] = accdeci[number]
        dataframe['Random'][item] = accrand[number]
    return dataframe        

In [256]:
dataset = pd.read_csv('prep.csv', index_col = None)  # index_col - keep the default index not allowing the input to reset the index position
dataset

,age,bp,sg,al,su,rbc,pc,pcc,ba,bgr,...,pcv,wc,rc,htn,dm,cad,appet,pe,ane,classification
0,2.000000,76.459948,c,3.0,0.0,normal,abnormal,notpresent,notpresent,148.112676,...,38.868902,8408.191126,4.705597,no,no,no,yes,yes,no,yes
1,3.000000,76.459948,c,2.0,0.0,normal,normal,notpresent,notpresent,148.112676,...,34.000000,12300.000000,4.705597,no,no,no,yes,poor,no,yes
2,4.000000,76.459948,a,1.0,0.0,normal,normal,notpresent,notpresent,99.000000,...,34.000000,8408.191126,4.705597,no,no,no,yes,poor,no,yes
3,5.000000,76.459948,d,1.0,0.0,normal,normal,notpresent,notpresent,148.112676,...,38.868902,8408.191126,4.705597,no,no,no,yes,poor,yes,yes
4,5.000000,50.000000,c,0.0,0.0,normal,normal,notpresent,notpresent,148.112676,...,36.000000,12400.000000,4.705597,no,no,no,yes,poor,no,yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
394,51.492308,70.000000,a,0.0,0.0,normal,normal,notpresent,notpresent,219.000000,...,37.000000,9800.000000,4.400000,no,no,no,yes,poor,no,yes
395,51.492308,70.000000,c,0.0,2.0,normal,normal,notpresent,notpresent,220.000000,...,27.000000,8408.191126,4.705597,yes,yes,no,yes,poor,yes,yes
396,51.492308,70.000000,c,3.0,0.0,normal,normal,notpresent,notpresent,110.000000,...,26.000000,9200.000000,3.400000,yes,yes,no,poor,poor,no,yes
397,51.492308,90.000000,a,0.0,0.0,normal,normal,notpresent,notpresent,207.000000,...,38.868902,8408.191126,4.705597,yes,yes,no,yes,poor,yes,yes


In [257]:
df1 = dataset
df1 = pd.get_dummies(df1, dtype = int, drop_first = True)

In [258]:
df1

,age,bp,al,su,bgr,bu,sc,sod,pot,hrmo,...,pc_normal,pcc_present,ba_present,htn_yes,dm_yes,cad_yes,appet_yes,pe_yes,ane_yes,classification_yes
0,2.000000,76.459948,3.0,0.0,148.112676,57.482105,3.077356,137.528754,4.627244,12.518156,...,0,0,0,0,0,0,1,1,0,1
1,3.000000,76.459948,2.0,0.0,148.112676,22.000000,0.700000,137.528754,4.627244,10.700000,...,1,0,0,0,0,0,1,0,0,1
2,4.000000,76.459948,1.0,0.0,99.000000,23.000000,0.600000,138.000000,4.400000,12.000000,...,1,0,0,0,0,0,1,0,0,1
3,5.000000,76.459948,1.0,0.0,148.112676,16.000000,0.700000,138.000000,3.200000,8.100000,...,1,0,0,0,0,0,1,0,1,1
4,5.000000,50.000000,0.0,0.0,148.112676,25.000000,0.600000,137.528754,4.627244,11.800000,...,1,0,0,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
394,51.492308,70.000000,0.0,0.0,219.000000,36.000000,1.300000,139.000000,3.700000,12.500000,...,1,0,0,0,0,0,1,0,0,1
395,51.492308,70.000000,0.0,2.0,220.000000,68.000000,2.800000,137.528754,4.627244,8.700000,...,1,0,0,1,1,0,1,0,1,1
396,51.492308,70.000000,3.0,0.0,110.000000,115.000000,6.000000,134.000000,2.700000,9.100000,...,1,0,0,1,1,0,0,0,0,1
397,51.492308,90.000000,0.0,0.0,207.000000,80.000000,6.800000,142.000000,5.500000,8.500000,...,1,0,0,1,1,0,1,0,1,1


In [259]:
indep_x = df1.drop('classification_yes', axis = 1)
dep_y = df1['classification_yes']

In [290]:
kbest = selectkbest(indep_x, dep_y, 7)

acclog = []
accsvml = []
accsvmnl = []
accknn = []
accnaive = []
accdeci = []
accrand = []

In [291]:
x_train, x_test, y_train, y_test = split_scaler(kbest, dep_y)

In [292]:
classifier,accuracy,report,x_test,y_test,cm = logistic(x_train,y_train,x_test)
acclog.append(accuracy)

classifier, accuracy, report, x_test, y_test, cm = svm_linear(x_train, y_train, x_test)
accsvml.append(accuracy)

classifier, accuracy, report, x_test, y_test, cm = svm_nl(x_train, y_train, x_test)
accsvmnl.append(accuracy)

classifier, accuracy, report, x_test, y_test, cm = knn(x_train, y_train, x_test)
accknn.append(accuracy)

classifier, accuracy, report, x_test, y_test, cm = naive(x_train, y_train, x_test)
accnaive.append(accuracy)

classifier, accuracy, report, x_test, y_test, cm = decision(x_train, y_train, x_test)
accdeci.append(accuracy)

classifier, accuracy, report, x_test, y_test, cm = random(x_train, y_train, x_test)
accrand.append(accuracy)

In [293]:
result = selectk_classification(acclog, accsvml, accsvmnl, accknn, accnaive, accdeci, accrand)

In [289]:
# For K=3
result

,Logistic,SVML,SVMNL,KNN,Naive,Decision,Random
ChiSquare,0.791667,0.808333,0.825,0.833333,0.783333,0.883333,0.85


In [283]:
# For K=4
result

,Logistic,SVML,SVMNL,KNN,Naive,Decision,Random
ChiSquare,0.858333,0.841667,0.833333,0.858333,0.791667,0.891667,0.908333


In [278]:
# For K=5
result

,Logistic,SVML,SVMNL,KNN,Naive,Decision,Random
ChiSquare,0.941667,0.941667,0.95,0.908333,0.841667,0.958333,0.958333


In [273]:
# For K = 6
result

,Logistic,SVML,SVMNL,KNN,Naive,Decision,Random
ChiSquare,0.95,0.958333,0.95,0.933333,0.891667,0.95,0.983333


In [294]:
# K=7
result

,Logistic,SVML,SVMNL,KNN,Naive,Decision,Random
ChiSquare,0.966667,0.966667,0.966667,0.958333,0.916667,0.966667,0.983333
